<a href="https://colab.research.google.com/github/Mudryi/investigat-opticalflow-attack/blob/main/optical_attack_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
!pip install numpy==1.15.4 

In [5]:
import torch

print(torch.__version__)

1.10.0+cu111


In [6]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html


In [7]:
!git clone https://github.com/open-mmlab/mmflow.git
%cd mmflow

fatal: destination path 'mmflow' already exists and is not an empty directory.
/content/mmflow


In [8]:
!pip install -r requirements/build.txt

In [9]:
!pip install -v -e .

Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin
sysconfig: /usr/bin
Value for scheme.data does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local
sysconfig: /usr
Additional context:
user = False
home

In [10]:
from mmflow.apis import inference_model, init_model
from mmflow.datasets import visualize_flow, write_flow
import matplotlib.pyplot as plt

import os
import tqdm

from PIL import Image, ImageDraw
import numpy as np

In [85]:
device = 'cuda:0'

path_to_data = '/content/gdrive/MyDrive/ucu_ml_research/data'
file_list = os.listdir(path_to_data)

paths = {'flownet2':['configs/flownet2/flownet2css_8x1_sfine_flyingthings3d_subset_384x768.py',
                     '/content/gdrive/MyDrive/ucu_ml_research/pwcnet_ft_4x1_300k_sintel_final_384x768.pth'],
         'pwc':['/content/mmflow/configs/pwcnet/pwcnet_ft_4x1_300k_sintel_384x768.py',
                '/content/gdrive/MyDrive/ucu_ml_research/pwcnet_ft_4x1_300k_sintel_final_384x768.pth'],
         'raft':['/content/mmflow/configs/raft/raft_8x2_100k_mixed_368x768.py',
                     '/content/gdrive/MyDrive/ucu_ml_research/raft_8x2_100k_mixed_368x768.pth'],
         'flownet':['/content/mmflow/configs/flownet/flownetc_8x1_sfine_sintel_384x448.py',
                     '/content/gdrive/MyDrive/ucu_ml_research/flownetc_8x1_sfine_sintel_384x448.pth']}

In [86]:
def load_model(model_name):
  model_paths = paths[model_name]
  config_file = model_paths[0]
  checkpoint_file = model_paths[1]
  model = init_model(config_file, checkpoint_file, device=device)
  return model

In [105]:
model = load_model('flownet2')

clear_image = np.zeros((150,150,3))
pos = (200, 200)

def gauss(mean,sigma):
     from random import uniform
     from math import sqrt,log,pi,cos
     a=uniform(0,1)
     b=uniform(0,1)
     x=sqrt(-2*log(a))*cos(2*pi*b)
     return(x)

def bruiter(image):
  a,b,c=image.shape
  for i in range(a):
    for j in range(b):
      image[i][j] += [gauss(0.5,0.01),gauss(0.5,0.01),gauss(0.5,0.01)]
  image = abs(np.round(image*50)).astype(np.uint8)
  return(image)

def add_noise_to_img(image, noise, pos = pos):
  image[pos[0]:pos[0]+noise.shape[0], pos[1]:pos[1]+noise.shape[1]] = noise
  return image

2022-01-29 13:35:39,037 - mmflow - INFO - Freeze the parameters in FlowNetC
2022-01-29 13:35:39,457 - mmflow - INFO - Freeze the parameters in FlowNetS


load checkpoint from local path: /content/gdrive/MyDrive/ucu_ml_research/pwcnet_ft_4x1_300k_sintel_final_384x768.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: encoder.layers.0.layers.0.conv.weight, encoder.layers.0.layers.0.conv.bias, encoder.layers.0.layers.1.conv.weight, encoder.layers.0.layers.1.conv.bias, encoder.layers.0.layers.2.conv.weight, encoder.layers.0.layers.2.conv.bias, encoder.layers.1.layers.0.conv.weight, encoder.layers.1.layers.0.conv.bias, encoder.layers.1.layers.1.conv.weight, encoder.layers.1.layers.1.conv.bias, encoder.layers.1.layers.2.conv.weight, encoder.layers.1.layers.2.conv.bias, encoder.layers.2.layers.0.conv.weight, encoder.layers.2.layers.0.conv.bias, encoder.layers.2.layers.1.conv.weight, encoder.layers.2.layers.1.conv.bias, encoder.layers.2.layers.2.conv.weight, encoder.layers.2.layers.2.conv.bias, encoder.layers.3.layers.0.conv.weight, encoder.layers.3.layers.0.conv.bias, encoder.layers.3.layers.1.conv.w

In [106]:
def calculate_metric(a,b):
  for i, j in zip(a,b):
    i[pos[0]:pos[0]+clear_image.shape[0], pos[1]:pos[1]+clear_image.shape[1]] = 0
    j[pos[0]:pos[0]+clear_image.shape[0], pos[1]:pos[1]+clear_image.shape[1]] = 0

  difference_array = np.subtract(a, b)
  squared_array = np.square(difference_array)
  return squared_array.mean()
  
def get_results(draw = False, use_noise = False, save_path = None):
  results = []

  if use_noise:
    noise = bruiter(clear_image)

  for i in range(len(file_list)-1):
    img = Image.open(path_to_data+'/'+file_list[i])
    img2 = Image.open(path_to_data+'/'+file_list[i+1])

    if draw:
      draw = ImageDraw.Draw(img)
      draw.rectangle((pos, (pos[0]+clear_image.shape[0], pos[1]+clear_image.shape[1])), fill="green")

      draw = ImageDraw.Draw(img2)
      draw.rectangle((pos, (pos[0]+clear_image.shape[0], pos[1]+clear_image.shape[1])), fill="green")

    img = np.asarray(img)
    img2 = np.asarray(img2)

    if use_noise:
      img.setflags(write=1)
      img2.setflags(write=1)

      img = add_noise_to_img(img, noise)
      img2 = add_noise_to_img(img2, noise)
    
    result = inference_model(model, img, img2)
    results.append(result)
    if save_path:
      visualize_flow(result, save_file=f'/content/gdrive/MyDrive/ucu_ml_research/{save_path}/img_{i}.png')

  return results

In [114]:
clear_results = get_results(save_path='flownet2')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


In [110]:
mse_list = []
for i in tqdm.trange(10):
  noised_results = get_results(use_noise=True, save_path = 'flownet2_noised')

  mse_list.append(calculate_metric(noised_results, clear_results)
)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 10/10 [13:31<00:00, 81.12s/it]


In [111]:
np.mean(mse_list)

0.0007075362

In [92]:
square_results = get_results(draw = True, save_path = 'flownet2_square')
calculate_metric(clear_results, square_results)


73.69141

In [117]:
import cv2
import os

image_folder = '/content/gdrive/MyDrive/ucu_ml_research/data'
image_folder_noised = '/content/gdrive/MyDrive/ucu_ml_research/pwc_noised'
image_folder_clear = '/content/gdrive/MyDrive/ucu_ml_research/pwc_clear'

video_name = '/content/gdrive/MyDrive/ucu_ml_research/pwc_examle.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images_noised = [img for img in os.listdir(image_folder_noised) if img.endswith(".png")]
images_clear = [img for img in os.listdir(image_folder_clear) if img.endswith(".png")]

frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 15, (width,height*3))

for image, noise, clear in zip(images, images_noised, images_clear):

  img_clear = cv2.imread(os.path.join(image_folder_clear, clear))
  img_ori = cv2.imread(os.path.join(image_folder, image))
  img_noised = cv2.imread(os.path.join(image_folder_noised, noise))

  vis = np.concatenate((img_ori, img_clear, img_noised), axis=0)
  video.write(vis)

cv2.destroyAllWindows()
video.release()